# Re-download previous Planet orders to file using the order ID

In [ ]:
# -----Define where to download items
out_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/write-ups/CH1_snow_cover_mapping_methods_manuscript/Aberle_et_al_dataset_submission/SouthCascade/imagery/PlanetScope/' 

# -----Import packages
import os
from getpass import getpass
from planet import Auth, Session, OrdersClient, reporting
import requests
from requests.auth import HTTPBasicAuth
import json
import pandas as pd
from pathlib import Path

### Authenticate Planet account

Requires your Planet API Key. To find your API Key, Login to your account at [planet.com](https://www.planet.com/) and go to "My Settings". 

If output is `<Repsonse [200]>`, authentication was successful. 

In [ ]:
# set API key as environment variable
API_key = getpass('Enter Planet API Key, then press Enter:')
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

### View list of previous Planet orders

In [ ]:
# grab list of orders from orders_url
orders = json.loads(response.text)['orders']
# initialize dataframe for saving order characteristics
orders_df = pd.DataFrame() 
# loop through orders
for order in orders:
    # compile order characteristics in dataframe
    order_df = pd.DataFrame({'Name': [order['name']],
                             'ID': [order['id']],
                             'Created on': [order['created_on']],
                             'Item type': [order['products'][0]['item_type']],
                             'Product bundle': [order['products'][0]['product_bundle']],
                             'Number of images': len(order['products'][0]['item_ids']),
                             'Last message': [order['last_message']]
                            })
    # concatenate to full dataframe
    orders_df = pd.concat([orders_df, order_df])
    
orders_df.reset_index(drop=True, inplace=True)
orders_df

### Select the ID of the order you want to download using the table above.

Orders can also be viewed on your Planet account in the Orders tab. 

In [ ]:
order_id = input('Enter order ID:')
order_id

### Download all items in the order to out_path

This may take a few minutes... be patient. :) You should see files being added in your `out_path`.

In [ ]:
async def download_order():
    async with Session(auth=auth) as sess:
        cl = OrdersClient(sess)
        await cl.download_order(order_id, directory=Path(out_path))
        
# remember: "await" to run the thing
await download_order()

### _Fin!_